# Set-up

In [1]:
# Imports
import os
import glob
import pandas as pd
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [3]:
# Paths
path_sample_metadata = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/metadata/sample_metadata.tsv"
path_out = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/metadata/2024_08_16"

In [4]:
# make output directory
os.makedirs(path_out, exist_ok=True)

# Metadata

In [5]:
df = pd.read_csv(path_sample_metadata, sep="\t")
df.head()

,sample_id,condition,timepoint,rep,harmonized_sample_id,year,cell_line,differentiation_batch,sequencing_batch,multiome_stage,multiome_qc_status,notes
0,mo1,control,0,fail,NaN,1,H1,DM023,NaN,annotated,fail,not using H1 samples
1,mo3,control,0,fail,NaN,1,H1,DM023,NaN,annotated,fail,not using H1 samples
2,dm0b,control,0,fail,NaN,2,A2,DM041,NaN,annotated,fail,"clustered separately in integration, still has..."
3,0-2,control,0,1,control_0_1,3,A2,JE002,NaN,annotated,pass,0-1 wasn't sequenced due to not enough cells
4,0-3B-1,control,0,2,control_0_2,3,A2,JE002,NaN,generated,not generated,NaN


In [7]:
# Filter out the samples that have "preprocessed" in "multiome_analysis_stage"
stages = "all"
if stages == "all":
    df_processed = df.copy()
else:
    df_processed = df[df["multiome_stage"].isin(stages)].reset_index(drop=True)
len(df_processed)

56

# `H5AD` files

In [8]:
# Get all the directories with fragment files in them
sample_dir = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/3_sample_qc"
rna_h5_paths = sorted(glob.glob(os.path.join(sample_dir, "**", "seurat_default_pca.h5ad"), recursive=True))
atac_h5_paths = sorted(glob.glob(os.path.join(sample_dir, "**", "clustered.h5ad"), recursive=True))
len(rna_h5_paths), len(atac_h5_paths)

(56, 54)

In [9]:
# Get dict of sample_id to path
sample_id_to_rna_path = {}
for input_h5ad_path in rna_h5_paths:
    sample_id = input_h5ad_path.split("/")[-4]
    sample_id_to_rna_path[sample_id] = input_h5ad_path

# Get dict of sample_id to path
sample_id_to_atac_path = {}
for input_h5ad_path in atac_h5_paths:
    sample_id = input_h5ad_path.split("/")[-3]
    sample_id_to_atac_path[sample_id] = input_h5ad_path

In [10]:
# h5ad and frag to df
df_processed["rna_h5_path"] = df_processed["sample_id"].map(sample_id_to_rna_path)
df_processed["atac_h5_path"] = df_processed["sample_id"].map(sample_id_to_atac_path)

In [11]:
# Which samples are currently missing rna_h5_path
missing_rna_h5_path = df_processed[df_processed["rna_h5_path"].isnull()]
missing_rna_h5_path["sample_id"].to_list()

[]

In [12]:
# Which samples are currently missing atac_h5_path
missing_atac_h5_path = df_processed[df_processed["atac_h5_path"].isnull()]
missing_atac_h5_path["sample_id"].to_list()

['0-2', '12-1']

In [26]:
# Drop nas
df_processed = df_processed.dropna(subset=["rna_h5_path", "atac_h5_path"])
len(df_processed)

10

In [27]:
# Save the output
df_processed[["rna_h5_path", "atac_h5_path", "sample_id", "condition", "differentiation_batch", "timepoint"]].sort_values(["condition", "timepoint"]).to_csv(os.path.join(path_out, "6_joint-integrate.tsv"), sep="\t", index=False, header=False)

# Annotation

In [36]:
def write_script(sample, outpath):
    script_contents="#!/bin/bash\n" \
    "script_path=/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/sample_annotation/scripts/cellcommander_annotate_RNA.sh\n" \
    f"sample={sample}\n" \
    "input_h5mu=/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/$sample/joint/integrate/joint_integrate.h5mu\n" \
    "outdir_path=/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/$sample/joint/annotate\n" \
    "cmd='bash $script_path $input_h5mu $outdir_path'\n" \
    "echo -e 'Running command: $cmd'\n" \
    "eval $cmd\n"
    with open(outpath, "w") as f:
        f.write(script_contents)

In [37]:
annot_path = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/sample_annotation/7_annotate"
for sample_id in sorted(df_processed["sample_id"].values):
    print(sample_id)
    write_script(sample_id, outpath=os.path.join(annot_path, f"7_cellcommander_annotate_{sample_id}.sh"))

14-1
24-1
24-2
31-1
34-1
35-1
41-1
41-2
44-1
45-1


# DONE!

---

In [ ]:
# print out when all the rna_h5_paths were generated
for rna_h5_path in df_processed["rna_h5_path"]:
    os.system(f"ls -l --time=ctime {rna_h5_path}")

-rw-r----- 1 aklie carter-users 362798557 Jul 29 09:04 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/35-1/rna/reduce_dimensions/seurat_default_pca.h5ad
-rw-r----- 1 aklie carter-users 218892049 Jul 29 09:07 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/45-1/rna/reduce_dimensions/seurat_default_pca.h5ad
-rw-r----- 1 aklie carter-users 602119845 Jul 29 09:06 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/14-1/rna/reduce_dimensions/seurat_default_pca.h5ad
-rw-r----- 1 aklie carter-users 614338877 Jul 29 09:17 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-1/rna/reduce_dimensions/seurat_default_pca.h5ad
-rw-r----- 1 aklie carter-users 645155157 Jul 29 09:06 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-2/rna/reduce_dimensions/seurat_default_pca.h5ad
-rw-r----- 1 aklie carter-users 572

In [ ]:
# print out when all the atac_h5_paths were generated
for atac_h5_path in df_processed["atac_h5_path"]:
    os.system(f"ls -l --time=ctime {atac_h5_path}")

-rw-r----- 1 aklie carter-users 1002870179 Jul 30 14:08 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/35-1/atac/clustered.h5ad
-rw-r----- 1 aklie carter-users 924850832 Jul 30 14:08 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/45-1/atac/clustered.h5ad
-rw-r----- 1 aklie carter-users 1500846303 Jul 30 14:45 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/14-1/atac/clustered.h5ad
-rw-r----- 1 aklie carter-users 2298322352 Jul 30 14:52 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-1/atac/clustered.h5ad
-rw-r----- 1 aklie carter-users 1846312554 Jul 30 14:42 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-2/atac/clustered.h5ad
-rw-r----- 1 aklie carter-users 2407083114 Jul 30 14:53 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/34-1/atac/clustere

In [ ]:
for sample_id in df_processed["sample_id"]:
    file = f"/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/{sample_id}/joint/integrate/joint_integrate.h5mu"
    os.system(f"ls -l --time=ctime {file}")

-rw-r----- 1 aklie carter-users 863069348 Jul 31 10:10 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/35-1/joint/integrate/joint_integrate.h5mu
-rw-r----- 1 aklie carter-users 478335908 Jul 31 10:09 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/45-1/joint/integrate/joint_integrate.h5mu
-rw-r----- 1 aklie carter-users 1580597260 Jul 31 10:11 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/14-1/joint/integrate/joint_integrate.h5mu
-rw-r----- 1 aklie carter-users 808452096 Jul 31 10:12 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-1/joint/integrate/joint_integrate.h5mu
-rw-r----- 1 aklie carter-users 1865657476 Jul 31 10:11 /cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/24-2/joint/integrate/joint_integrate.h5mu
-rw-r----- 1 aklie carter-users 1640121820 Jul 31 10:11 /cellar/users/aklie/da

In [ ]:
# Convert the specified date to a timestamp
from datetime import datetime
date_cutoff = datetime.strptime('2024-08-10', '%Y-%m-%d').timestamp()
complete = 0
incomplete = 0
for sample_id in sorted(df_processed["sample_id"].values):
    print(sample_id)
    file = f"/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/sample_annotation/{sample_id}/joint/annotate/annotate.h5mu"
    if os.path.exists(file):
        file_mod_time = os.path.getmtime(file)
        if file_mod_time < date_cutoff:
            print(sample_id, datetime.fromtimestamp(file_mod_time))
            incomplete += 1
        else:
            complete += 1
print(f"Complete: {complete}")
print(f"Incomplete: {incomplete}")